In [1]:
import numpy as np
import pandas as pd
import pyMuellerMat
from pyMuellerMat.common_mm_functions import *
from pyMuellerMat import common_mms as cmm
from pyMuellerMat import MuellerMat
from scipy.optimize import minimize
import copy
from collections import defaultdict
from pathlib import Path

In [14]:
def add_fake_stds(csv_path):
    """Adds stds, 5% of the diff then 5% of the sum
    to the csv files.

    Parameters:
    -----------
    csv_path : str or Path
        Path to the CSV file to be modified.
    
    Returns:
    -----------
    None
    """
    csv_path = Path(csv_path)
    
    # Read the CSV file
    df = pd.read_csv(csv_path)

    # Calculate the new standard deviations
    diff_stds = df["norm_single_diff"] * 0.05
    sum_stds = df["single_sum"] * 0.05

    # Add the new columns to the DataFrame
    df["diff_std"] = diff_stds
    df["sum_std"] = sum_stds

    # Save the modified DataFrame back to the CSV file
    df.to_csv(csv_path, index=False)



In [4]:
from instruments import read_csv
from instruments import write_fits_info_to_csv

In [5]:
# write csvs for all wavelength bins
for bin in range(0,22):
    write_fits_info_to_csv(cube_directory_path='/Users/thomasmcintosh/Desktop/CHARIS-REU/Cubes/cubes/internal-cal',raw_cube_path='/Users/thomasmcintosh/Desktop/CHARIS-REU/Raw/internal-cal',output_csv_path=f'/Users/thomasmcintosh/Desktop/CHARIS-REU/Data_Visualization/csvs/cube_info_bin{bin}.csv',wavelength_bin=bin)

CSV file written to /Users/thomasmcintosh/Desktop/CHARIS-REU/Data_Visualization/csvs/cube_info_bin0.csv
CSV file written to /Users/thomasmcintosh/Desktop/CHARIS-REU/Data_Visualization/csvs/cube_info_bin1.csv
CSV file written to /Users/thomasmcintosh/Desktop/CHARIS-REU/Data_Visualization/csvs/cube_info_bin2.csv
CSV file written to /Users/thomasmcintosh/Desktop/CHARIS-REU/Data_Visualization/csvs/cube_info_bin3.csv
CSV file written to /Users/thomasmcintosh/Desktop/CHARIS-REU/Data_Visualization/csvs/cube_info_bin4.csv
CSV file written to /Users/thomasmcintosh/Desktop/CHARIS-REU/Data_Visualization/csvs/cube_info_bin5.csv
CSV file written to /Users/thomasmcintosh/Desktop/CHARIS-REU/Data_Visualization/csvs/cube_info_bin6.csv
CSV file written to /Users/thomasmcintosh/Desktop/CHARIS-REU/Data_Visualization/csvs/cube_info_bin7.csv
CSV file written to /Users/thomasmcintosh/Desktop/CHARIS-REU/Data_Visualization/csvs/cube_info_bin8.csv
CSV file written to /Users/thomasmcintosh/Desktop/CHARIS-REU/Dat

In [10]:
# add fake stds to all csvs
for bin in range(0,22):
    add_fake_stds(f'/Users/thomasmcintosh/Desktop/CHARIS-REU/Data_Visualization/csvs/cube_info_bin{bin}_fixed.csv')

In [6]:
# fix hwp angles
from instruments import fix_hwp_angles
for bin in range(0,22):
    fix_hwp_angles(f'/Users/thomasmcintosh/Desktop/CHARIS-REU/Data_Visualization/csvs/cube_info_bin{bin}.csv')

Fixed HWP angles saved to /Users/thomasmcintosh/Desktop/CHARIS-REU/Data_Visualization/csvs/cube_info_bin0_fixed.csv
Fixed HWP angles saved to /Users/thomasmcintosh/Desktop/CHARIS-REU/Data_Visualization/csvs/cube_info_bin1_fixed.csv
Fixed HWP angles saved to /Users/thomasmcintosh/Desktop/CHARIS-REU/Data_Visualization/csvs/cube_info_bin2_fixed.csv
Fixed HWP angles saved to /Users/thomasmcintosh/Desktop/CHARIS-REU/Data_Visualization/csvs/cube_info_bin3_fixed.csv
Fixed HWP angles saved to /Users/thomasmcintosh/Desktop/CHARIS-REU/Data_Visualization/csvs/cube_info_bin4_fixed.csv
Fixed HWP angles saved to /Users/thomasmcintosh/Desktop/CHARIS-REU/Data_Visualization/csvs/cube_info_bin5_fixed.csv
Fixed HWP angles saved to /Users/thomasmcintosh/Desktop/CHARIS-REU/Data_Visualization/csvs/cube_info_bin6_fixed.csv
Fixed HWP angles saved to /Users/thomasmcintosh/Desktop/CHARIS-REU/Data_Visualization/csvs/cube_info_bin7_fixed.csv
Fixed HWP angles saved to /Users/thomasmcintosh/Desktop/CHARIS-REU/Data_

In [13]:
# try read csv func
from instruments import read_csv
sys_dict = read_csv('/Users/thomasmcintosh/Desktop/CHARIS-REU/Data_Visualization/csvs/cube_info_bin0_fixed.csv')[2]
sys_dict 


[{'hwp': {'theta': 0.0}, 'image_rotator': {'theta': 45.0}},
 {'hwp': {'theta': 11.25}, 'image_rotator': {'theta': 45.0}},
 {'hwp': {'theta': 22.5}, 'image_rotator': {'theta': 45.0}},
 {'hwp': {'theta': 33.75}, 'image_rotator': {'theta': 45.0}},
 {'hwp': {'theta': 45.0}, 'image_rotator': {'theta': 45.0}},
 {'hwp': {'theta': 56.25}, 'image_rotator': {'theta': 45.0}},
 {'hwp': {'theta': 67.5}, 'image_rotator': {'theta': 45.0}},
 {'hwp': {'theta': 78.75}, 'image_rotator': {'theta': 45.0}},
 {'hwp': {'theta': 90.0}, 'image_rotator': {'theta': 45.0}},
 {'hwp': {'theta': 0.0}, 'image_rotator': {'theta': 57.5}},
 {'hwp': {'theta': 11.25}, 'image_rotator': {'theta': 57.5}},
 {'hwp': {'theta': 22.5}, 'image_rotator': {'theta': 57.5}},
 {'hwp': {'theta': 33.75}, 'image_rotator': {'theta': 57.5}},
 {'hwp': {'theta': 45.0}, 'image_rotator': {'theta': 57.5}},
 {'hwp': {'theta': 56.25}, 'image_rotator': {'theta': 57.5}},
 {'hwp': {'theta': 67.5}, 'image_rotator': {'theta': 57.5}},
 {'hwp': {'theta': 